Introduction:
The basis for our code comes from David Pollard's 1982 paper, A simple ice sheet model yields realistic 100 glacial cycles. Pollard wanted to build of the previous work of Birchfield et al. and Oerlemans in order to produce an accurate model of the 100 kyr ice cycle. Pollard's goal was to create a model that that would closely follow the records of ice volume in glaciers for the past 700 kyr. Pollard's model differs from other previous models in the fact that it takes into account both topography and possible calving. These additions led to a model that matched the 700 kyr records more closely than any other model up to that point in time. It's important that scientists try to get their models to closely follow past data, because it means that said models will likely be more accurate in predicting the future. Glaciers are an incredibly important part of the earth’s climate system, and as global temperatures rise it has become clear that glaciers are shrinking at an alarming rate. The ability to accurately model glaciers is more important now than ever due to the effects of climate change. The main numerical method used in the paper is the Newton-Raphson method, which is applied to two equations separately. TALK ABOUT OTHER MODELS

Equations: 
Northern hemisphere glacial cycles are quite complex and depend on quite a lot of variables. From historical records it can be observed that there is a dominant cycle of glacial ice volume that occurs over an 100 kyr period. The main equation used by Pollard is a simplified diffusion equation for ice thickness which is as follows:

$$\frac{\partial h}{\partial t} = A \frac{\partial}{\partial x}\left[h^{\alpha} \left|\frac{\partial (h + h')}{\partial x}\right|^{\beta} \frac{\partial (h + h')}{\partial x}\right] + G(h + h', x, \text{orbit})$$

This is quite a long formula, but once each term is defined it becomes much easier to understand. The variable t is time while x is the distance from north to south. h' is defined as the elevation of the bedrock surface above a fixed point, which in this case is the mean sea level from 1982. This formula is also made up of some constants, A is one such constant that is equal to 5.77e-4 1/m^3*yr. Alpha and beta are also constants, equal to 5 and 2 respectively. The northern boundary of the model is 74° N while the southern boundary is taken to be 30° N. At these points h = 0 which is to prevent ice formation in the arctic and tropics. G is the net annual mass balance on the ice surface and is dependent on a separate set of equations from Oerlemans(SITE): 

$$
G = \begin{cases}
a(h + h' - E) - b(h + h' - E)^2 & \text{if } h + h' - E \leq 1{,}500 \text{ m} \\
0.56 & \text{if } h + h' - E > 1{,}500 \text{ m}
\end{cases} \text{ m yr}^{-1}
$$



This equation also is built upon some constants, being a and b. a is equal to 0.81 * 10^-3 1/yr, while b is 0.30 * 10^-6 1/m*yr. E is the equilibrium altitude, and is the value that controls the value of G. E has a constant slope and is dependent on its own equation E = E0(x) + k * dQ. E0 is the present equilibrium line and dQ is the difference in the summer half year insolation from 55° N from the present, which in this case is 1982. K is the isolation sensitivity and like E0, is a value that is changed each run to yield different results.

There is a third main equation that is used in this one-dimensional model, and this equation is related to the change in bedrock due to the change in ice load. The Pollard states that at the time of the paper there wasn’t a consensus on what kind of model would be best for this, so he chose to use a “thin-channel model”. The equation is as follows:

$$\frac{\partial h'}{\partial t} = \nu \frac{\partial^2}{\partial x^2}[h' - h'_0(x) + rh]$$

This equation seems to be quite simple; however, the "v" term in the equation is itself quite complicated. V is equal to ρa * g * H^3 / Nη, with the simplest of these variables being g, the gravitational constant one the surface of earth. H is the mean thickness along a channel while η is the dynamic viscosity of said channel. N is either 3 or 12 depending on the requirements at the base of the lithosphere. r is the ratio of ice density (ρi) to rock density (ρa). h’0(x) is the surface topography that would occur without ice, and is the variable that determines the boundary conditions for this equation. Pollard chose to have h’ = h’0 such that the latitudinal bounds are 74° N and 30° N. 

The main variables that will be modified between each run are the parameters that Pollard modified, those being, E0, k, r, v, and h'0(x). The main limitation of these eqautions is that they only account for ice thickness running north to south. Pollard rectifies this by multiplying by an east-west ice sheet dimenstion of about 3000 km to compute the volume. ADD MORE HERE.

Numerical Methods:
Need to elabolrate on this more, haven't had time while working on the code

In [ ]:
# Setup (Imports and Variables)
# NOTE: All units of distance are in METERS
# 1 degree latiture ~= 111,000 m
# Since all h values are in meters, it is better to use meters instead of degrees when solving\
# all relevant constant/intital values are stored in the values dictionary
# code DOES NOT RUN RIGHT NOW, need to fix intial values as well as the main loop, ice density function also needs a rework


import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.sparse import diags


def initialize(numberPoints, dt):

    values = {}

    values['n'] = numberPoints
    values['dt'] = dt # years

    values['alpha'] = 5
    values['beta'] = 2
    values['A'] = 5.77 * 10 ** -4
    values['lat_max'] = 74
    values['lat_min'] = 30

    values['a'] = 0.81 * 10 ** -3
    values['b'] = 0.30 * 10 ** -6
    values['r'] = 0.3
    values['k'] = 17
    values['nu'] = 100
    values['v'] = 100000 * 1000 # km2 / yr, converted to meters 

    values['dx'] = (74-30) / (numberPoints - 1)
    values['dx_m'] = values['dx'] * 111000


    values['h'] = np.zeros(numberPoints)
    values["h_prime"] = np.zeros(numberPoints)
    values['lat'] = np.linspace(30, 74, numberPoints)
    values['E0'] = 550 + 111 * (values['lat']-70) # from paper, this math might be wrong, will probably have to change
    values["h_prime0"] = np.interp(values['lat'], np.array([30, 40, 66, 70, 74]), np.array([400, 400, 200, 850, -500])) #generating topography

    return values

v = initialize(80, 50) # Test values from paper, 80 points and 50 year timesteps


In [ ]:
def bedrockM (size): #Creating the matricies that will be used to solve equation 3, DOES NOT NEED TO BE CALLED MORE THAN ONCE
    arr_size = size
    cD = v['v'] * v['dt'] / v['dx_m'] ** 2
    ML_Data = np.array([[-0.5*cD]*arr_size,[1+cD]*arr_size,[-0.5*cD]*arr_size])
    MLdiag = np.array([-1,0,1])
    ML = sp.sparse.spdiags(ML_Data, MLdiag, arr_size, arr_size).toarray()

    MR_Data = np.array([[0.5*cD]*arr_size,[1-cD]*arr_size,[0.5*cD]*arr_size])
    MRdiag = np.array([-1,0,1])
    MR = sp.sparse.spdiags(MR_Data, MRdiag, arr_size, arr_size).toarray()

    # Applying Dirichlet conditions to arrays
    # not sure why the entire row needs to be zero, had to look it up to get this function working, LOOK INTO THIS
    ML[0, :] = 0
    ML[0, 0] = 1
    MR[0, :] = 0
    MR[0, 0] = 1

    ML[arr_size-1, :] = 0
    ML[arr_size-1, arr_size-1] = 1
    MR[arr_size-1, :] = 0
    MR[arr_size-1, arr_size-1] = 1


    return ML, MR

In [ ]:
# Mass balance function, solving for G, likely not going to work first time, dQ is a mess to calculate so
# I wrote a placeholder function for now, will have more time when I'm not dying from exams to fix this.

def massBal(h, h_prime, timestep):
    dQ = 50 * np.sin(2*np.pi * timestep / 100000)
    elevation = h + h_prime
    G = np.zeros(len(elevation))
    E = v['E0'] + v['k'] * dQ
    for i in range(len(elevation)):
        if elevation[i] - E[i] <= 1500:
            #quadratic function
            G[i] = v['a'] * (elevation[i] - E[i]) - v['b'] * (elevation[i] - E[i]) ** 2
        elif elevation[i] - E[i] > 1500:
            #constant
            G[i] = 0.56
    
    return G

In [ ]:
def iceDensity(h, h_prime):
    noZeroMath = 1e-10 # prevents h and the gradient from being zero, cause that would cause a lot of problems 
    dx = v['dx_m']
    n = len(h)
    elevation = h + h_prime
    gradient = np.zeros(len(elevation))
    for i in range(1, n - 1):
        gradient[i] = (elevation[i + 1] - elevation[i-1]) / (2 * dx)
    
    # boundary conditions for gradient array
    gradient[0] = (elevation[1] - elevation[0]) / dx
    gradient[-1] = (elevation[-1] - elevation[-2]) / dx
    
    diffuse = v['A'] * (h + noZeroMath) ** v['alpha'] * (np.abs(gradient) + noZeroMath) ** v['beta']

    # Different way of creating the matricies than the bedrock due to the fact that we have to change them each timestep
    # we have to recalculate our "cD" each time
    lower = np.zeros(n - 1)
    mid = np.ones(n)
    upper = np.zeros(n - 1)
    
    for i in range(1, n-1):
        D_left = 0.5 * (diffuse[i-1] + diffuse[i])
        D_right = 0.5 * (diffuse[i] + diffuse[i+1])
        
        r_left = 0.5 * v['dt'] * D_left / dx ** 2
        r_right = 0.5 * v['dt'] * D_right / dx ** 2
        
        lower[i-1] = -r_left
        mid[i] = 1 + r_left + r_right
        upper[i] = -r_right

    # creating ML
    mid[0] = 1
    upper[0] = 0
    lower_full = np.zeros(n)
    lower_full[1:] = lower

    mid_full = mid

    upper_full = np.zeros(n)
    upper_full[:-1] = upper

    ML_Data = np.vstack([lower_full, mid_full, upper_full])
    MLdiag = np.array([-1,0,1])
    ML = sp.sparse.diags(ML_Data, MLdiag, shape=(n,n)).toarray()

    # had to look this up, idk how it works to be honest need to figure that out
    lower_MR = -lower
    mid_MR = 2 - mid
    upper_MR = -upper
    

    lower_MR_full = np.zeros(n)
    lower_MR_full[1:] = lower_MR

    mid_MR_full = mid_MR

    upper_MR_full = np.zeros(n)
    upper_MR_full[:-1] = upper_MR
    MR_data = [lower_MR, mid_MR, upper_MR]
    MRdiag = np.array([-1,0,1])
    MR = sp.sparse.diags(MR_data, MRdiag, shape=(n,n)).toarray()
    
    MR[0, :] = 0
    MR[0, 0] = 1
    
    return ML, MR



In [ ]:
def solveIceEquation(h, h_prime, time):
    ML_Ice, MR_Ice = iceDensity(h, h_prime) # fix this function
    G = massBal(h, h_prime, time) # change timestep if needed idk if this is right
    RHS = MR_Ice @ h + v['dt'] * G
    # need to apply boundary conditions 
    RHS[0] = 0
    h_updated = np.linalg.inv(ML_Ice) @ RHS  # this math might be wrong, check later
    return h_updated

In [ ]:
def solveBedrock(h, h_prime):
    ML_Bedrock, MR_Bedrock = bedrockM(v['n'])
    temp_var = h_prime - v['h_prime0'] + v['r'] * h # makes matrix math easier than it would be otherwise
    temp_var[0] = 0
    temp_var[-1] = 0
    temp_new = np.linalg.inv(ML_Bedrock) @ MR_Bedrock @ temp_var
    # do we need to apply boundary conditions here? maybe later
    final_hPrime = temp_new + v['h_prime0'] - v['r'] * h
    final_hPrime[0] = v['h_prime0'][0]
    final_hPrime[-1] = v['h_prime0'][-1]
    return final_hPrime

In [ ]:
# setting up the results arrays
n_years = 700000 /v['dt']


save_interval = 50

numSave = int(n_years // save_interval) + 1

results_h = np.zeros((numSave, v['n']))
results_h_prime = np.zeros((numSave, v['n']))
results_time = np.zeros(numSave)
results_volume = np.zeros(numSave)

current_time = 0.0
save_index = 0

# main loop, saving all the info from running the functions
for step in range(numSave):
    time = step * v['dt']
    
    h_old = v['h'].copy()
    h_prime_old = v['h_prime'].copy()
    
    h_new = solveIceEquation(h_old, h_prime_old, current_time)
    
    h_avg = 0.5 * (h_old + h_new)
    
    h_prime_new = solveBedrock(h_avg, h_prime_old)
    
    v['h'] = h_new
    v['h_prime'] = h_prime_new
    
    if step % save_interval == 0:
        results_h[save_index, :] = v['h']
        results_h_prime[save_index, :] = v['h_prime']
        results_time[save_index] = time
        
        results_volume[save_index] = np.trapz(v['h'], dx=v['dx_m']) * 3000 / 1e9  # from paper, gets total volume

        save_index += 1



# Seems like the functions work! maybe the numbers are wrong and thats why the plot isn't correct at all

plt.figure(figsize=(12, 5))
plt.plot(results_time, results_volume, 'b-', linewidth=2)
plt.show()

The code still isn't completely functional so we don't have any results to discuss yet